# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
# imports

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [4]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [ ]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [5]:
# Get gpt-4o-mini to answer, with streaming
from IPython.display import display, Markdown
import openai

# Your model
MODEL = "gpt-4o"  
# System prompt (sets the assistant's role)
system_prompt = "You are a helpful Python expert who explains code clearly and concisely."

# The user question
question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

# Streaming function for explanation
def stream_code_explanation(code_question):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": code_question}
        ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    
    for chunk in stream:
        content = chunk.choices[0].delta.content or ""
        response += content
        response_cleaned = response.replace("```", "").replace("markdown", "")
        display_handle.update(Markdown(response_cleaned))

# Run it!
stream_code_explanation(question)


Certainly! This line of code is using a combination of Python set comprehension and the `yield from` statement. Let's break it down step-by-step:

1. **Set Comprehension**: 
   - `{book.get("author") for book in books if book.get("author")}` is a set comprehension. It constructs a set containing the authors from the `books` collection.
   - `books` is assumed to be an iterable, such as a list or tuple, of dictionaries, where each dictionary represents a book with various attributes, including an "author".
   - `book.get("author")` attempts to retrieve the value associated with the "author" key from each book dictionary.
   - The `if book.get("author")` part ensures that only books with a non-None value for the "author" key are included in the set. This helps in filtering out entries where the "author" key doesn't exist or is set to `None`.

2. **Yield From**: 
   - `yield from` is a statement in Python that delegates part of a generator's operations to another iterable.
   - `yield from` in this context is used to yield each element from the set constructed by the set comprehension one at a time.

3. **Purpose**:
   - The combination of set comprehension and `yield from` results in yielding each distinct author found in the list of books, but importantly, without duplicates, since sets naturally eliminate duplicates.
   - This might be part of a generator function, which is a function allowing iteration over the unique authors.

In summary, this code is used to retrieve and yield each unique author's name from a list of book dictionaries, one by one, eliminating any duplicates. The use of `yield from` makes it efficient for building generators that process data as an iterable sequence.

In [ ]:
# Get Llama 3.2 to answer